In [1]:
import param
import panel as pn
import pandas as pd

from bokeh.sampledata.iris import flowers
from bokeh.sampledata.autompg import autompg_clean
from bokeh.sampledata.population import data

pn.extension()

This examples demonstrates the reactive programming paradigm using which uses the param library to express parameters and methods which express computation depending on those parameters. This pattern can be used to update the displayed views when a parameter value changes.

In [4]:
class Reactive(param.Parameterized):
    
    title = param.String(default='Data Summary')
    
    dataset = param.ObjectSelector(default='iris', objects=['iris', 'autompg', 'population'])
    
    rows = param.Integer(default=10, bounds=(0, 100))
    
    @param.depends('dataset')
    def data(self):
        if self.dataset == 'iris':
            return flowers
        elif self.dataset == 'autompg':
            return autompg_clean
        else:
            return data

    @param.depends('data')
    def summary(self):
        return self.data().describe()
    
    @param.depends('title')
    def header(self):
        return '##' + self.title
    
    @param.depends('data', 'rows')
    def table(self):
        return self.data().iloc[:self.rows]
    
    def panel(self):
        return pn.Row(
            self.param,
            pn.Column(
                self.header,
                self.summary,
                self.table
            ),
            min_height=1000
        )
    
reactive = Reactive()
reactive.panel()

Row(min_height=1000)
    [0] Column(margin=5, name='Param00062', width=300)
        [0] StaticText(value='<b>Reactive</b>')
        [1] TextInput(name='Title', value='Data Summary')
        [2] Select(name='Dataset', options=OrderedDict([('iris', ...]), value='iris')
        [3] IntSlider(end=100, name='Rows', value=10)
    [1] Column
        [0] ParamMethod(method)
        [1] ParamMethod(method)
        [2] ParamMethod(method)